# Dual Measure and Bucket Sort

Sort the blue marbles by value and put them in the leftmost column. Highest value on the bottom, Lowest value on top.
Sort the yellow marbles by value and put them in the rightmost column. Highest value on the bottom, Lowest value on top.
Initially the values are unknown random integers between **1** and **3** inclusive. Before you can read ball values via the **get_state()** function you must first measure each ball's value by dropping it on the magic spot (x=4, y=6).
Claw 0 can operate within the left half of the grid including the magic reveal spot in the middle column.
Claw 1 can operate within the right half of the grid including the magic reveal spot in the middle column.

In [ ]:
import asyncio
%pip install ballsort
from ballsort.ballsort_display_utils import open_bs_window
open_bs_window()

In [ ]:
from control_factory import get_control_sim
from ch11_scenario import Ch11Scenario

In [ ]:
from ball_control import BallControl
from state_update_model import StatePosition

async def move_ball(bc: BallControl, src: StatePosition, dest: StatePosition, claw_index: int = 0):
    rel_x = src.x - bc.get_position(claw_index=claw_index).x
    rel_y = src.y - bc.get_position(claw_index=claw_index).y
    await asyncio.gather(
        bc.move_horizontally(rel_x, claw_index=claw_index),
        bc.move_vertically(rel_y, claw_index=claw_index),
        bc.open_claw(claw_index=claw_index))
    await bc.close_claw(claw_index=claw_index)
    
    rel_x = dest.x - bc.get_position(claw_index=claw_index).x
    rel_y = dest.y - bc.get_position(claw_index=claw_index).y
    await asyncio.gather(
        bc.move_horizontally(rel_x, claw_index=claw_index),
        bc.move_vertically(rel_y, claw_index=claw_index))
    await bc.open_claw(claw_index=claw_index)

async def move_ball_by_column(bc: BallControl, src_x: int, dest_x: int, claw_index: int = 0):
    src_column_top_occupied_y = min([ball.pos.y for ball in bc.get_state().balls if ball.pos.x == src_x],default=bc.get_state().max_y)
    dest_column_top_vacant_y = min([ball.pos.y for ball in bc.get_state().balls if ball.pos.x == dest_x],default=bc.get_state().max_y + 1) - 1
    await move_ball(bc=bc, src=StatePosition(x=src_x, y=src_column_top_occupied_y), dest=StatePosition(x=dest_x, y=dest_column_top_vacant_y), claw_index=claw_index)

In [ ]:
from ball_control_sim import BallControlSim

async def bucket_sort(
    bc: BallControlSim,
    dest_x: int,
    bucket_offset: int,
    claw_index: int,
    lock: asyncio.Lock,
):
    reveal_spot_x = bc.get_state().max_x // 2
    reveal_spot_y = bc.get_state().max_y

    # reveal each ball's value by moving it to the revealer spot. Then move it to bucket column.
    for _ in range(len(bc.get_state().balls) // 2):
        async with lock:
            await move_ball_by_column(
                bc=bc, src_x=dest_x, dest_x=reveal_spot_x, claw_index=claw_index
            )  # reveal
            revealed_value = next(
                (
                    ball.value
                    for ball in bc.get_state().balls
                    if ball.pos == StatePosition(x=reveal_spot_x, y=reveal_spot_y)
                ),
                None,
            )
            assert revealed_value
            await move_ball_by_column(
                bc=bc,
                src_x=reveal_spot_x,
                dest_x=bucket_offset + revealed_value,
                claw_index=claw_index,
            )  # move to column corresponding to value

    # Finally move all buckets to destination column in the correct order
    for x in range(bucket_offset + 3, bucket_offset + 0, -1):
        nof_balls_in_column = len(
            [ball for ball in bc.get_state().balls if ball.pos.x == x]
        )
        for _ in range(nof_balls_in_column):
            await move_ball_by_column(
                bc=bc, src_x=x, dest_x=dest_x, claw_index=claw_index
            )

In [ ]:
async def challenge11_solution():
    bc = get_control_sim(delay_multiplier=0.1)
    await bc.set_scenario(Ch11Scenario(seed=None))
    
    lock = asyncio.Lock() #necessary to enforce mutually exclusive access to the revealer column

    await asyncio.gather(
        bucket_sort(bc=bc, dest_x=0, bucket_offset=0, claw_index=0, lock=lock),
        bucket_sort(
            bc=bc,
            dest_x=bc.get_state().max_x,
            bucket_offset=bc.get_state().max_x // 2,
            claw_index=1,
            lock=lock,
        ),
    )

    assert bc.get_state().goal_accomplished

In [ ]:
await challenge11_solution()